# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/google/magenta-realtime',
 '/nanonets/Nanonets-OCR-s',
 '/mistralai/Mistral-Small-3.2-24B-Instruct-2506',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/OmniGen2/OmniGen2',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/multimodalart/self-forcing',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/nvidia/AceReason-1.1-SFT',
 '/datasets/BAAI/CCI4.0-M2-CoT-v1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transfo

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The 

In [23]:
system_prompt = "You are an humorous assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'Contact/Community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ngoogle/magenta-realtime\nUpdated\n3 days ago\n•\n342\nnanonets/Nanonets-OCR-s\nUpdated\n6 days ago\n•\n177k\n•\n1.19k\nmistralai/Mistral-Small-3.2-24B-Instruct-2506\nUpdated\n4 days ago\n•\n5.37k\n•\n255\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n2 days ago\n•\n10.4k\n•\n586\nOmniGen2/OmniGen2\nUpdated\n3 days ago\n•\n1.15k\n•\n180\nBrowse 1M+ models\nSpaces\nRunning\n834\n834\nSparc3D\n🏃\nNext-G

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to Hugging Face

## **The AI Community Building the Future**

Hugging Face is at the forefront of the artificial intelligence and machine learning revolution, providing a collaborative platform where enthusiasts, developers, and enterprises come together to create, discover, and deploy machine learning models, datasets, and applications.

### **What We Offer**
- **Models**: Access and contribute to over 1 million state-of-the-art machine learning models across various domains including text, image, video, audio, and 3D.
- **Datasets**: Explore and utilize our extensive collection of over 250,000 datasets specifically curated for a wide array of machine learning tasks.
- **Spaces**: Run and share applications in your browser, enabling easy collaboration and sharing among developers and researchers.

### **Our Impact**
More than **50,000 organizations** trust Hugging Face. Notable clients include industry giants like **Google**, **Amazon**, **Microsoft**, and **Meta**, all leveraging our open-source ecosystem to democratize machine learning.

### **Our Community Spirit**
- **Open Source Collaboration**: We actively engage the community in building foundational tools for machine learning, exemplified by our diverse projects such as Transformers, Diffusers, and Tokenizers.
- **Supportive Environment**: We pride ourselves on fostering a welcoming and inclusive community where every member contributes to the advancement of ML technologies.

### **Careers at Hugging Face**
Join a passionate team of over **213 members** committed to **democratizing machine learning**. We welcome individuals driven by curiosity and eager to make a meaningful impact in the AI space. Explore our job opportunities at [Hugging Face Careers](https://huggingface.co/jobs).

### **Company Culture**
- **Mission-Driven**: Our mission is to democratize access to machine learning technologies.
- **Collaborative**: Work alongside talented individuals who are enthusiastic about sharing knowledge and innovating collaboratively.
- **Flexible and Inclusive**: Embrace a workplace that values diversity and inclusivity in all its forms.

---

### **Join Us!**
Whether you're a developer, a business, or a job seeker, Hugging Face welcomes all to be a part of this innovative journey. 

- **Explore AI Apps**: [Explore](https://huggingface.co/models)
- **Date & Features**: Stay updated with the latest trends in the community.

**Connect with us:**
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)

Join us today as we build the future of AI together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## **About Us**
**Hugging Face** is at the forefront of the artificial intelligence revolution, uniting the global machine learning community to collaborate on models, datasets, and innovative applications. We believe in the power of open-source collaboration and aim to build the foundation of machine learning (ML) tooling together with the community.

### **Our Mission**
To accelerate innovation in AI by providing the tools, platform, and support necessary for researchers and developers to create, discover, and publish high-quality machine learning applications.

---

## **What We Offer**

### **Our Platform:**
- **Models & Datasets**: We host over 1 million models and 250,000 datasets, empowering users to create powerful AI solutions.
- **Spaces**: A collaborative environment where you can experiment and showcase AI applications in various modalities like text, images, videos, audio, and even 3D models.
- **Compute Solutions**: Paid options for optimized inference endpoints and enterprise-grade solutions designed for extensive scalability.

### **Enterprise Solutions**
Hugging Face serves more than 50,000 organizations globally, including giants like Google, Microsoft, Intel, and Amazon. Our enterprise solutions include:
- Advanced platform features for building AI with enhanced security and access controls.
- Dedicated support and customization to fit team needs, starting at $20/user/month. 

---

## **Community**
We pride ourselves on our vibrant and inclusive community where anyone can contribute. Our repositories, including **Transformers**, **Diffusers**, and **Tokenizers**, rank among the top ML tools globally.

### **Notable Collaborators**
- **AI at Meta**
- **Grammarly**
- **Writer**
- **AI2**

Join us in building cutting-edge technology for machine learning!

---

## **Company Culture**
At Hugging Face, we foster a dynamic and inclusive culture that thrives on collaborative spirit, diversity of thought, and innovation. Our team is driven by passion and commitment to advancing AI responsibly and ethically. 

We encourage ongoing learning and growth, and aim to build a supportive environment where every voice is valued.

---

## **Careers**
We are constantly on the lookout for talented individuals who share our vision for democratizing AI. Explore our current openings and discover how you can contribute to a diverse team dedicated to making a meaningful impact in technology.

**Current Openings**: [View Careers](#)

---

## **Connect with Us**
Stay connected through our social channels:

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

Join us in shaping the future of artificial intelligence at Hugging Face!

--- 

### **Get Started Today!**
Whether you’re an individual developer, a team at an enterprise, or a researcher looking to collaborate, Hugging Face has something for you. Sign up today to start exploring the vast world of machine learning!

[Sign Up Now](#)

---

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## Welcome to Hugging Face!

### The AI Community Building the Future 🤖✨

At Hugging Face, we believe that artificial intelligence is the key to unlocking a smarter future. Our platform serves as a bustling hub for the machine learning community, where collaboration, innovation, and creativity come together to produce cutting-edge models, datasets, and applications. 

---

## Who We Are

**The AI Wizards:** We are a passionate team of engineers, researchers, and AI enthusiasts who are striving to make machine learning accessible for everyone! Our mission? To democratize AI and foster a supportive community where knowledge is shared, and everyone thrives.

**Company Culture:** 
- **Collaborative Spirit:** We cherish the enthusiasm of our community and actively encourage contributions. 
- **Inclusivity:** Everyone is welcome to join the conversation. No wizardry degree needed—just curiosity!
- **A Dash of Humor:** We believe that laughter is the best algorithm. Expect friendly memes buzzing around our virtual space!

---

## Platforms & Products

### Models, Datasets, and Spaces 🛠️ 

- **1M+ Models:** Discover, develop, and share models with a thriving community of creators. Popular models include:
  - `google/magenta-realtime`
  - `nanonets/Nanonets-OCR-s`
  - `mistralai/Mistral-Small-3.2-24B-Instruct-2506`

- **Datasets:** Access over **250k datasets** tailored for every machine learning task imaginable. 

- **Spaces:** Run applications seamlessly with our robust platform, featuring real-time video generation and image-to-3D capabilities.

---

## Customer Base 🌍

With over **50,000 organizations** leveraging Hugging Face, our clientele includes some of the biggest names in tech:
- Google
- Amazon
- Microsoft
- Intel

Whether you're a startup, an enterprise, or a non-profit, there's a space for you at our table!

---

## Career Opportunities 🚀

Join us on this wild ride through the world of AI! We're always on the lookout for:
- **Engineers** who dream in code.
- **Data Scientists** who can kiss datasets and make them sing.
- **Community Managers** who can spark conversations and keep the magic alive.

**Perks of Working With Us:** 
- Flexible work hours (better than a magic wand!).
- Opportunities for growth and learning.
- A community that truly cares for one another (and we have great snacks!).

---

🚀 **Join Us!** Ready to unlock the potential of AI with a community driven by passion and innovation? Explore our platform, dive into our resources, and become part of the Hugging Face family today!

**Contact Us:** [Hugging Face Website](https://huggingface.co)

---
*Give us a wave and watch as your AI aspirations take flight!*

This brochure succinctly captures the essence of Hugging Face, highlighting company culture, products, customers, and career opportunities while maintaining a light-hearted tone.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>